### 0050成分股獲利數據抓取

In [1]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import datetime as dt

### 鉅亨網 > 0050成分股清單抓取

In [85]:
url = 'https://www.cnyes.com/twstock/ingredient.aspx?code=0050'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://fubon-ebrokerdj.fbs.com.tw/',
    'Accept-Language': 'en-US,en;q=0.9',
}
res_j = re.get(url, headers=headers)
soup = BeautifulSoup(res_j.text, 'html.parser')

rows = soup.find_all('table', {'id':'TB'})[0].find_all('tr')
sybq=[]
for i in range(1, len(rows)):
  rowsd = rows[i].find_all('td', {'class':'alignl'})
  for j in range(len(rowsd)):
    sybq.append(rowsd[j].text)

print(sybq)


['1101', '1102', '1216', '1301', '1303', '1326', '1402', '2002', '2105', '2207', '2301', '2303', '2308', '2317', '2327', '2330', '2357', '2382', '2395', '2408', '2412', '2454', '2474', '2633', '2801', '2823', '2880', '2881', '2882', '2883', '2884', '2885', '2886', '2887', '2888', '2890', '2891', '2892', '2912', '3008', '3045', '3711', '4904', '4938', '5871', '5876', '5880', '6505', '9904', '9910']


### 富邦證券 > 0050經營績效資料抓取

In [86]:
dfc = pd.DataFrame()

for i in sybq:

  # 用beautifulsoup 抓取網頁原始碼
  url = 'https://fubon-ebrokerdj.fbs.com.tw/z/zc/zce/zce_' + str(i) + '.djhtm'
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
      'Referer': 'https://fubon-ebrokerdj.fbs.com.tw/',
      'Accept-Language': 'en-US,en;q=0.9',
  }
  res_j = re.get(url, headers=headers)
  soup = BeautifulSoup(res_j.text, 'html.parser')


  # 抓取資料所在的tag
  rows = soup.find_all('tr')
  sybn = rows[3].find_all('td', {'class':'t10'})[0].text.split(')')[0] + ')'

  lst_t = []
  for r in rows[5:6]:
    lst = []
    for d in r.find_all('td'):
      lst.append(d.text)
    lst_t.append(lst)

  lst_t = lst_t[0]

  lst_c = []
  for r in rows[6:-2]:
    lst = []
    for d in r.find_all('td'):
      lst.append(d.text)
    lst_c.append(lst)

  lst_all = []
  for i in lst_c:
    dd={}
    for j in range(len(i)):
      dd[lst_t[j]]=i[j]
    lst_all.append(dd)

  df = pd.DataFrame(lst_all)
  df.insert(0, '股票名稱', sybn)

  # 彙整資料成為dataframe格式
  dfc = pd.concat([dfc, df], axis=0)
dfc.reset_index(inplace=True, drop=True) 

### 資料整理(文字轉數值)

In [89]:
dff = dfc.copy()
dff[dff.columns[2:]] = dff[dff.columns[2:]].astype(str).replace({',':'', '%':'', '':np.nan}, regex=True).astype(float)
dff['毛利率'] = dff['毛利率']*0.01
dff['營益率'] = dff['營益率']*0.01

### 資料整理(季別轉日期)

In [97]:
def chageDate(value):
    if isinstance(value, str):
        if '1Q' in value:
            yrs = int(value.split('.')[0]) + 1911 
            mon = 5
            dat = 15
            return dt.date(yrs, mon, dat)

        if '2Q' in value:
            yrs = int(value.split('.')[0]) + 1911 
            mon = 8
            dat = 14
            return dt.date(yrs, mon, dat)

        if '3Q' in value:
            yrs = int(value.split('.')[0]) + 1911 
            mon = 11
            dat = 14
            return dt.date(yrs, mon, dat)

        if '4Q' in value:
            yrs = int(value.split('.')[0]) + 1911 + 1
            mon = 3
            dat = 31
            return dt.date(yrs, mon, dat)

dff['季別'] = dff['季別'].map(chageDate)

### 資料匯出

In [2]:
print(dff.head(20).to_string())
dff.to_csv(os.path.join(os.getcwd() , '0050成分股_獲利能力.csv'), encoding='utf-8-sig')

NameError: name 'dff' is not defined